# Scrape Donuts Store Information

### We will scrape donuts store information within New York Area
### Our url is : 'https://www.yelp.com/search?find_desc=donuts&find_loc=New%20York%2C%20NY&ns=1&cflt=donuts'

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.request import urlopen
from selenium import webdriver

In [10]:
original_url = 'https://www.yelp.com/search?find_desc=donuts&find_loc=New%20York%2C%20NY&cflt=donuts'
file_path = 'C:/Users/yukun/Desktop/DonutsStores/NYDonuts_page.html'
for i in range(0,1):#since we have tried before, the last page that we are able to generate is page#34
    path = file_path + str(i+1) + '.html'
    url = original_url + '&start=' + str(i*30)
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)
    source = driver.page_source
    soup = BeautifulSoup(source, 'html.parser')
    file = open(path, mode = 'w', encoding = 'utf-8')
    file.write(soup.prettify())
    file.close()
    driver.close()

In [13]:
#We could pull out the page source from local drive
html_path = 'C:/Users/yukun/Desktop/DonutsStores/NYDonuts_page.html'
data = []
for i in range(1,34):
    path = html_path + str(i) + '.html'
    file = open(path, mode = 'r', encoding = 'utf-8')
    soup = BeautifulSoup(file, 'html.parser')
    stores = soup.find_all('li',{'class':'lemon--li__373c0__1r9wz border-color--default__373c0__2oFDT'})
    for store in stores:
    #Firstly find out the store names and store href by following
        store_name_tag = store.find('a','lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5')
        if store_name_tag is not None:
            store_href = store_name_tag['href']
            #Remove the advertisement information
            if not store_href.startswith('/adredir'):
                store_name = store_name_tag.get_text()
                store_href1 = store_href
                #Find the store rating
                store_rating_tag1 = store.find('span','lemon--span__373c0__3997G display--inline__373c0__1DbOG border-color--default__373c0__2oFDT')
                if store_rating_tag1 is not None:
                    store_rating_tag = store_rating_tag1.find('div')
                    if store_rating_tag is not None:
                        store_rating = store_rating_tag['aria-label'].replace('star rating','')
                #Find total reviews
                store_total_reviews_tag = store.find('div','lemon--div__373c0__1mboc attribute__373c0__1hPI_ display--inline-block__373c0__2de_K border-color--default__373c0__2oFDT')
                if store_total_reviews_tag is not None:
                    store_reviews = store_total_reviews_tag.get_text().replace('reviews','')
                #Find store phone number
                phone_number_tag1 = store.find('div','lemon--div__373c0__1mboc container__373c0__19wDx u-padding-l2 border-color--default__373c0__2oFDT text-align--right__373c0__3fmmn')
                if phone_number_tag1 is not None:
                    phone_number_tag2 = phone_number_tag1.find('p','lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--right__373c0__3ARv7')
                    if phone_number_tag2 is not None:
                        phone_number = phone_number_tag2.get_text()
                #Find the location
                store_location_tag1 = store.find('div','lemon--div__373c0__1mboc container__373c0__19wDx u-padding-l2 border-color--default__373c0__2oFDT text-align--right__373c0__3fmmn')
                if store_location_tag1 is not None:
                    store_location_tag2 = store_location_tag1.find('span','lemon--span__373c0__3997G')
                    if store_location_tag2 is not None:
                        store_location = store_location_tag2.get_text()
                #Find the neighbor
                store_neighbor_tag1 = store.find('div','lemon--div__373c0__1mboc u-space-b1 border-color--default__373c0__2oFDT')
                if store_neighbor_tag1 is not None:
                    store_neighbor_tag2 = store_neighbor_tag1.find('p','lemon--p__373c0__3Qnnj text__373c0__2pB8f text-color--normal__373c0__K_MKN text-align--right__373c0__3ARv7')
                    if store_neighbor_tag2 is not None:
                        store_neighbor = store_neighbor_tag2.get_text()
                data.append([store_name, store_href1, store_rating, store_reviews, phone_number, store_location, store_neighbor])

In [25]:
for i in range(0,len(data)):
    for x in range(0,7):
        data[i][x] = data[i][x].strip()

In [28]:
with open('C:/Users/yukun/Desktop/DonutsStores/Donuts_store_info.csv', mode = 'w', encoding = 'utf-8') as file:
    file.write('store_name'+','+'store_link'+','+'store_rating'+','+'number_of_reviews'+','+'store_phone#'+','+'store_address'+','+'store_neighbor'+'\n')
    for k in data:
        file.write(k[0]+','+k[1]+','+k[2]+','+k[3]+','+k[4]+','+k[5]+','+k[6]+'\n')

## Preprocess Data

In [36]:
df = pd.read_csv('C:/Users/yukun/Desktop/DonutsStores/Donuts_store_info.csv')
df = df.dropna()
df['number_of_reviews'] = df['number_of_reviews'].str.replace('review','')
df['number_of_reviews'] = df['number_of_reviews'].astype(int)
df.to_csv('C:/Users/yukun/Desktop/DonutsStores/Donuts_store_info2.csv')